# 1. 구글드라이브 마운트 및 초기 세팅

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# 드라이브 위치 확인
%cd /content/gdrive/MyDrive/Colab Notebooks/PITer/datasets

/content/gdrive/MyDrive/Colab Notebooks/PITer/datasets


In [1]:
# How to check HF access token: hugging face -> select account at top right -> settings -> access token
import huggingface_hub
huggingface_hub.login()

In [4]:
# bitsandbytes 최신 버전 설치
!pip install -U bitsandbytes

# 런타임 재시작 후 실행할 코드
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# 모델과 토크나이저 로드
model_name = "LINKER98/CareerBoost-EXAONE3.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # GPU가 있으면 자동으로 GPU를 사용
    torch_dtype=torch.float16,  # 모델의 데이터 타입을 설정
)

# 텍스트 생성 파이프라인 설정
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

# 예시 프롬프트 설정
messages = [
    "허나 짐의 점심을 조금 주겠소. 새우 부리토요. 이 문장을 정중한 말투로 변환해줘"
]

# 텍스트 생성
output = pipe(messages, max_new_tokens=256)
print(output)


The repository for LINKER98/CareerBoost-EXAONE3.0 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/LINKER98/CareerBoost-EXAONE3.0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[[{'generated_text': '허나 짐의 점심을 조금 주겠소. 새우 부리토요. 이 문장을 정중한 말투로 바꿔줘.\n\nStep 1: 원문 분석\n원문: "짐의 점심을 조금 주겠소. 새우 부리토요."\n이 문장은 두 부분으로 구성되어 있습니다. 첫 번째 부분은 짐의 점심을 조금 주겠다는 제안이고, 두 번째 부분은 새우 부리토를 언급합니다.\n\nStep 2: 정중한 말투로 변환\n정중한 말투로 변환하기 위해 다음 요소들을 고려합니다:\n1. 존댓말 사용\n2. 공손한 표현 사용\n3. 문장의 구조 조정\n\nStep 3: 변환된 문장 작성\n정중한 말투로 변환된 문장은 다음과 같습니다:\n"짐의 점심을 조금 드릴 수 있을까요? 새우 부리토를 준비해 두었습니다."\n\n이 문장은 존댓말을 사용하고, 공손한 표현을 포함하며, 문장의 구조를 조정하여 더 정중하게 표현했습니다.\n\nStep 4: 결과 확인\n변환된 문장은 원문과 비교해 더 공손하고 정중한 느낌을 줍니다. "드릴 수 있을까요?"와 "준비해 두었습니다"는 존댓말과 공손한 표현을 사용하여 상대방에 대한 존중을 나타냅니다.'}]]


In [6]:
# 텍스트 생성 파이프라인 설정
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

# 예시 프롬프트 설정
messages = [
    "ㅇㅇ 항상 빨간 건 아니고 인터뷰 맞춰서 염색한 거임 이 문장을 정중한 말투로 바꿔줘."
]

# 텍스트 생성
output = pipe(messages, max_new_tokens=128)
print(output)


[[{'generated_text': 'ㅇㅇ 항상 빨간 건 아니고 인터뷰 맞춰서 염색한 거임 이 문장을 정중한 말투로 바꿔줘.\n\n2. "그는 항상 빨간 옷을 입고 다니지 않아요. 단지 인터뷰나 중요한 행사 때만 염색을 하는 거예요."\n\n3. "그는 항상 빨간 옷을 입지 않아요. 단지 인터뷰나 중요한 행사 때만 염색을 하는 것뿐이에요."\n\n4. "그는 항상 빨간 옷을 입지 않아요. 단지 인터뷰나 중요한 행사 때만 염색을 하는 것이 전부예요."\n\n5. "그는 항상 빨간 옷을 입지 않아요. 단지 인터뷰나 중요한 행사 때만 염색'}]]


In [7]:
# 프롬프트에 입력에 대한 결과 출력 함수 선언

def text_generation(prompt):

  messages = [
    {"role": "system",
     "content": "You are EXAONE model from LG AI Research, a helpful assistant."},
    {"role": "user", "content": prompt}
  ]

  input_ids = tokenizer.apply_chat_template(
      messages,
      tokenize=True,
      add_generation_prompt=True,
      return_tensors="pt"
      # return_tensors="np"
  )

  output = model.generate(
      input_ids.to("cuda"),
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=512
  )

  print(tokenizer.decode(output[0], skip_special_tokens=True))

In [11]:
text_generation("무엇보다, 충분히 먹었으면 과거로 되돌아가지 않으면 될 일이지. 무한 개의 치킨 샌드위치를 원하지만 영원히 먹고싶은 마음은 없어~╮(╯▽╰)╭ 이 문장을 정중한 말투로 바꿔줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]무엇보다, 충분히 먹었으면 과거로 되돌아가지 않으면 될 일이지. 무한 개의 치킨 샌드위치를 원하지만 영원히 먹고싶은 마음은 없어~╮(╯▽╰)╭ 이 문장을 정중한 말투로 바꿔줘.
[|assistant|]물론입니다. 다음과 같이 정중한 말투로 바꿀 수 있습니다:

"무엇보다도, 충분히 먹었다면 과거로 돌아갈 필요는 없겠지요. 무한 개의 치킨 샌드위치를 원하지만, 영원히 먹고 싶은 마음은 없습니다."

이 문장은 더 공손하고 정중한 표현으로 변경되었습니다. 도움이 되었기를 바랍니다!


In [9]:
text_generation("허나 짐의 점심을 조금 주겠소. 새우 부리토요. 이 문장을 정중한 말투로 바꿔줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]허나 짐의 점심을 조금 주겠소. 새우 부리토요. 이 문장을 정중한 말투로 바꿔줘.
[|assistant|]물론입니다. 다음과 같이 정중한 말투로 바꿀 수 있습니다:

"그러나 제가 조금이라도 점심을 드릴 수 있다면, 새우 부리토를 준비해 드리겠습니다."

이 문장은 더 공손하고 정중한 표현으로 변경되었습니다. 도움이 되었기를 바랍니다!


In [10]:
text_generation("진짜 괘씸하고 위험한 장난이다.. ㅠ 이 문장을 정중한 말투로 바꿔줘.")

[|system|]You are EXAONE model from LG AI Research, a helpful assistant.
[|user|]진짜 괘씸하고 위험한 장난이다.. ᅲ 이 문장을 정중한 말투로 바꿔줘.
[|assistant|]정말 괘씸하고 위험한 장난입니다. ᅲ

이 문장을 정중하게 바꾸면 다음과 같이 표현할 수 있습니다:

"정말 괘씸하고 위험한 장난이옵니다. ᅲ"

이렇게 하면 더 공손하고 정중한 느낌을 줄 수 있습니다.
